In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [4]:
df = pd.read_csv("transformed_data/merged_data_1.csv", index_col="id")
cities = df['city'].unique().tolist()

In [5]:
df["room_type"].value_counts()

room_type
Entire home/apt    32648
Private room       18693
Shared room          366
Name: count, dtype: int64

In [6]:
df['num_bedrooms'].value_counts().sort_values(ascending=False)

num_bedrooms
1     36333
2      9290
0      4485
3      1477
4        96
5        10
9        10
6         2
10        2
8         2
Name: count, dtype: int64

In [7]:
# filter number of bedrooms by room_type = Entire home/apt and count them
df[df['room_type'] == 'Entire home/apt']['num_bedrooms'].value_counts()

num_bedrooms
1    18489
2     8650
0     4026
3     1410
4       67
5        4
6        2
Name: count, dtype: int64

In [8]:
df_copy = df[(df['room_type'] == 'Entire home/apt') & (df['num_bedrooms'] == 2)].copy()
df_copy

,price_total,room_type,is_shared_room,is_private_room,max_guests,is_superhost,is_multi_listing,is_business_listing,cleanliness_score,guest_satisfaction_score,...,longitude,latitude,city,day_type,district,state,country_code,country_name,Crime_Index,Safety_Index
id,,,,,,,,,,,,,,,,,,,,,
8,1001.804420,Entire home/apt,False,False,4.0,False,0,0,9.0,96.0,...,4.86459,52.40175,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
21,933.845757,Entire home/apt,False,False,4.0,False,0,0,10.0,96.0,...,4.89088,52.36422,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
28,1032.971668,Entire home/apt,False,False,4.0,False,1,0,9.0,96.0,...,4.92432,52.37676,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
32,516.017154,Entire home/apt,False,False,4.0,False,0,0,9.0,100.0,...,4.88881,52.36397,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
33,602.254353,Entire home/apt,False,False,3.0,False,0,0,10.0,100.0,...,4.86021,52.34502,Amsterdam,weekday,Gemeente Amsterdam,North Holland,NL,Netherlands,25.7,74.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,231.634060,Entire home/apt,False,False,3.0,False,1,0,7.0,90.0,...,16.35842,48.19046,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6
1777,168.758619,Entire home/apt,False,False,4.0,False,0,1,8.0,79.0,...,16.33940,48.21299,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6
1783,486.408153,Entire home/apt,False,False,6.0,True,0,0,10.0,99.0,...,16.39024,48.20891,Vienna,weekend,Wien Stadt,Vienna,AT,Austria,28.4,71.6


In [9]:
cities

['Amsterdam',
 'Athens',
 'Barcelona',
 'Berlin',
 'Budapest',
 'Lisbon',
 'London',
 'Paris',
 'Rome',
 'Vienna']

In [10]:
100/2 < 50

False

In [11]:
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# --- ROBUST RATING/FAV EXTRACTOR ---
def get_details_and_fav(driver, wait):
    rating, reviews, is_fav = "N/A", "0", 0 

    try:
        # Locate the review anchor (link or button)
        try:
            review_link = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//a[contains(@href, 'reviews')] | //button[contains(., 'reviews')]")
            ))
        except:
            # Fallback for "New" listings
            try:
                h1_parent = driver.find_element(By.TAG_NAME, "h1").find_element(By.XPATH, "./..").text
                if "New" in h1_parent: return "New", "0", 0
            except: pass
            return rating, reviews, is_fav

        # Get text content (cleanly)
        link_text = review_link.get_attribute("textContent").strip()
        clean_link = " ".join(link_text.split())
        
        # Get parent text (for standard listings where rating is outside link)
        parent_text = review_link.find_element(By.XPATH, "./..").get_attribute("textContent").strip()
        clean_parent = " ".join(parent_text.split())

        # --- SCENARIO A: GUEST FAVORITE ---
        if "Guest favorite" in clean_link or "Guest favorite" in clean_parent:
            is_fav = 1
            # Rating logic for fav
            r_match = re.search(r"(\d\.\d{1,2})", clean_parent)
            if r_match: rating = r_match.group(1)
            # Review logic for fav
            rev_match = re.search(r"(\d+)\s*Review", clean_parent, re.IGNORECASE)
            if rev_match: reviews = rev_match.group(1)

        # --- SCENARIO B: STANDARD ---
        else:
            is_fav = 0
            if "New" in clean_parent:
                rating = "New"
            else:
                r_match = re.search(r"(\d\.\d{1,2})", clean_parent)
                if r_match: rating = r_match.group(1)

            # Try link first, then parent for reviews
            rev_match = re.search(r"(\d+)\s*review", clean_link, re.IGNORECASE)
            if not rev_match:
                rev_match = re.search(r"(\d+)\s*review", clean_parent, re.IGNORECASE)
            if rev_match: reviews = rev_match.group(1)

    except: pass
    return rating, reviews, is_fav
all_data = []
city_count = 0

for city in cities:
    city_count += 1
    
    # --- MAIN SETUP ---
    URL = f"https://www.airbnb.com/s/{city}/homes?refinement_paths%5B%5D=%2Fhomes&date_picker_type=calendar&search_type=filter_change&flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2026-02-01&monthly_length=3&monthly_end_date=2026-05-01&price_filter_input_type=2&channel=EXPLORE&price_filter_num_nights=5&min_bedrooms=2&selected_filter_order%5B%5D=min_bedrooms%3A2&selected_filter_order%5B%5D=room_types%3AEntire%20home%2Fapt&update_selected_filters=true&query={city}&search_mode=regular_search&room_types%5B%5D=Entire%20home%2Fapt"

    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    target_count = 5
    # Initialize a local counter
    items_scraped_this_city = 0 
    try:
        driver.get(URL)
        wait = WebDriverWait(driver, 8)

        while items_scraped_this_city < target_count:
            print(f"\n=== SCRAPING CITY {city_count}/{len(cities)}: {city} ===\n")
            print(f"--- Scraping Page (Collected: {len(all_data)}) ---")
            
            try:
                wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div[data-testid='card-container']")))
            except: break

            listings = driver.find_elements(By.CSS_SELECTOR, "div[data-testid='card-container']")
            listing_urls = []
            for l in listings:
                try:
                    url = l.find_element(By.TAG_NAME, "a").get_attribute("href")
                    listing_urls.append(url)
                except: continue

            main_window = driver.current_window_handle

            for url in listing_urls:
                if items_scraped_this_city >= target_count: break
                
                try:
                    driver.execute_script(f"window.open('{url}', '_blank');")
                    driver.switch_to.window(driver.window_handles[-1])
                    wait.until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
                    
                    # A. Name
                    try: name = driver.find_element(By.TAG_NAME, "h1").text
                    except: name = "No Title"

                    # B. Superhost
                    is_superhost = "No"
                    try:
                        driver.execute_script("window.scrollBy(0, 500);")
                        time.sleep(0.5) 
                        body_text = driver.find_element(By.TAG_NAME, "body").text
                        if "Superhost" in body_text: is_superhost = "Yes"
                    except: pass

                    # C. Price (Robust)
                    price = driver.find_element(By.XPATH, "//div[@data-plugin-in-point-id='BOOK_IT_SIDEBAR']/div/div/div/div[1]/div[1]/div/div/span/div[1]/div/span/div/button/span").text
                    
                    # D. Rating/Fav (Robust)
                    rating, reviews, is_fav = get_details_and_fav(driver, wait)

                    all_data.append({
                        "Name": name, "city": city, "Price": price, "Superhost": is_superhost,
                        "is_fav": is_fav, "Rating": rating, "Reviews": reviews, "URL": url
                    })

                    print(f"[{len(all_data)}] {price} |{city}| ★{rating} ({reviews}) | Fav: {is_fav} | SH: {is_superhost}|")
                    time.sleep(random.uniform(1.0, 2.0))

                    driver.close()
                    driver.switch_to.window(main_window)
                    
                    # Increment the counter for successfully scraped items
                    items_scraped_this_city += 1
                    
                except Exception:
                    if len(driver.window_handles) > 1:
                        driver.close()
                        driver.switch_to.window(main_window)
                    continue

            if items_scraped_this_city < target_count:
                try:
                    next_btn = driver.find_element(By.CSS_SELECTOR, "a[aria-label='Next']")
                    driver.execute_script("arguments[0].click();", next_btn)
                    time.sleep(4)
                except: break

    finally:
        print(f"\nCompleted! Total scraped: {len(all_data)}")
        driver.quit()


=== SCRAPING CITY 1/10: Amsterdam ===

--- Scraping Page (Collected: 0) ---
[1] ج.م71,433 |Amsterdam| ★4.81 (592) | Fav: 1 | SH: Yes|
[2] ج.م140,461 |Amsterdam| ★5.0 (07) | Fav: 1 | SH: Yes|
[3] ج.م140,461 |Amsterdam| ★5.0 (07) | Fav: 1 | SH: Yes|
[4] ج.م170,305 |Amsterdam| ★5.0 (08) | Fav: 1 | SH: Yes|
[5] ج.م97,797 |Amsterdam| ★5.0 (010) | Fav: 1 | SH: Yes|

Completed! Total scraped: 5

=== SCRAPING CITY 2/10: Athens ===

--- Scraping Page (Collected: 5) ---

=== SCRAPING CITY 2/10: Athens ===

--- Scraping Page (Collected: 5) ---
[6] ج.م24,256 |Athens| ★4.93 (9375) | Fav: 1 | SH: Yes|
[7] ج.م15,766 |Athens| ★4.84 (84126) | Fav: 1 | SH: No|
[8] ج.م13,692 |Athens| ★4.91 (233) | Fav: 0 | SH: No|

=== SCRAPING CITY 2/10: Athens ===

--- Scraping Page (Collected: 8) ---
[9] ج.م20,191 |Athens| ★4.73 (79) | Fav: 0 | SH: Yes|
[10] ج.م27,898 |Athens| ★4.98 (98103) | Fav: 1 | SH: Yes|

Completed! Total scraped: 10

=== SCRAPING CITY 3/10: Barcelona ===

--- Scraping Page (Collected: 10) ---


In [12]:
df_scraped = pd.DataFrame(all_data)
df_scraped

,Name,city,Price,Superhost,is_fav,Rating,Reviews,URL
0,Stylish Serviced Apartment near Central Station,Amsterdam,"ج.م71,433",Yes,1,4.81,592,https://www.airbnb.com/rooms/30762441?search_m...
1,City Center Design Home,Amsterdam,"ج.م140,461",Yes,1,5.0,07,https://www.airbnb.com/rooms/13704561874549415...
2,Keizersgracht Canal House,Amsterdam,"ج.م140,461",Yes,1,5.0,07,https://www.airbnb.com/rooms/14823737915270235...
3,Dam Center Oasis,Amsterdam,"ج.م170,305",Yes,1,5.0,08,https://www.airbnb.com/rooms/14842444349648428...
4,2 Bedroom Luxury canal view,Amsterdam,"ج.م97,797",Yes,1,5.0,010,https://www.airbnb.com/rooms/65991245398895048...
5,Artistic Luxury-Acropolis View-Psirri/monastiraki,Athens,"ج.م24,256",Yes,1,4.93,9375,https://www.airbnb.com/rooms/13233428644375528...
6,Executive 2BD Acropolis Apartment,Athens,"ج.م15,766",No,1,4.84,84126,https://www.airbnb.com/rooms/87588765793315572...
7,Piraeus Port Suites 2 bedrooms 6 pax with balcony,Athens,"ج.م13,692",No,0,4.91,233,https://www.airbnb.com/rooms/12035290825858698...
8,Acropolis Sunlight 2BR Apartment,Athens,"ج.م20,191",Yes,0,4.73,79,https://www.airbnb.com/rooms/11411220989287776...
9,"2B/2B Modern & Family Friendly, walk to Acropo...",Athens,"ج.م27,898",Yes,1,4.98,98103,https://www.airbnb.com/rooms/11613966661221008...


In [14]:
df_scraped["city"].value_counts()

city
Amsterdam    5
Athens       5
Barcelona    5
Berlin       5
Lisbon       5
London       5
Rome         5
Name: count, dtype: int64